# **SIGN IDD: Iconicity Disentangled Diffusion for Sign Language Production**

# Initial imports and configuration

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#import packages
import os
import torch
import pickle
import numpy as np
import pandas as pd
import time
import shutil #for copying configs (directory operations)
import queue #for managing checkpoints

In [3]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
config_path = '/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Configs/Sign-IDD.yaml'
ckpt = '/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Checkpoints/final_model.pth'

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD')

In [ ]:
print(os.listdir('/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD'))
print("constants" in sys.modules)
import constants

['Checkpoints', 'loss.py', 'dtw.py', 'transformer_layers.py', 'Configs', '.ipynb_checkpoints', 'model_architecture.png', 'ACD.py', 'Data', 'Models', 'initialization.py', 'model_architecture', 'data.py', 'eval_helper.py', 'plot_videos.py', 'helpers.py', 'model.py', 'encoder.py', 'ID.py', 'prediction.py', 'torchtext_compat.py', 'embeddings.py', 'vocabulary.py', 'builders.py', '__pycache__', 'ACD_Denoiser (1).py', 'batch (1).py', 'constants (1).py', 'constants.py', 'ACD_Denoiser.py', 'batch.py', 'Main.ipynb']
False


In [6]:
%pip install tensorboardX
%pip install rouge_score
%pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6c7a4634a60cfb4afbb8449ddc79f4bc6911558a62bc27af7d8a6db14d155fb7
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.7 MB/s eta 0:00:00


In [7]:
#import custom modules
from torch import Tensor
import data
from data import load_data
import torchtext_compat
from torchtext_compat import make_data_iterator, Batch, TextField, RawField
#import batch
#from batch import Batch
from helpers import load_config, set_seed, load_checkpoint, log_cfg, make_model_dir, make_logger, ConfigurationError, get_latest_checkpoint, symlink_update
from model import build_model, Model
from tensorboardX import SummaryWriter
from constants import TARGET_PAD
from loss import Loss
from builders import build_gradient_clipper, build_optimizer, build_scheduler
from torch.utils.data import Dataset
from plot_videos import plot_video, alter_DTW_timing
from prediction import validate_on_data

# Train Manager Class

In [8]:
class TrainManager:
    def __init__(self, model: Model, config: dict, test= False):
        train_config= config["training"]  #load the configs specific to training from the yaml file
        model_dir= train_config["model_dir"]  #get the dir where model checkpoints and logs are stored
        model_continue = train_config.get("continue", True)  #continue from the latest checkpoint if true

        #if directory does not exist, cannot continue from anything, if in test mode, force continuation
        if not os.path.isdir(model_dir):
            model_continue = False
        if test:
            model_continue = True

        self.use_cuda = False

        #create model dir if it doesnt exist
        self.model_dir = make_model_dir(train_config["model_dir"],
                                overwrite=train_config.get("overwrite", False),
                                model_continue=model_continue)

        #logger setup
        self.logger = make_logger(model_dir=self.model_dir)
        self.logging_freq = train_config.get("logging_freq", 100)
        self.tb_writer = SummaryWriter(log_dir=self.model_dir+"/tensorboard/")
        self.valid_report_file = "{}/validations.txt".format(self.model_dir)  #file where validation scores will be written

        #model
        self.model = model  #store model
        self.pad_index = self.model.pad_index
        self.bos_index = self.model.bos_index
        self._log_parameters_list()  #log model parameters for reference

        #initialise loss object with loss function (L1 loss) and padding info
        self.target_pad = TARGET_PAD
        self.loss = Loss(cfg=config, target_pad=self.target_pad)

        #do gradient normalisation per batch
        self.normalization = "batch"

        #optimisation settings
        self.learning_rate_min = train_config.get("learning_rate_min", 1.0e-8)
        self.clip_grad_fun = build_gradient_clipper(config=train_config)
        self.optimizer = build_optimizer(config=train_config, parameters=model.parameters())  #adam

        #validation and early stopping
        self.validation_freq = train_config.get("validation_freq", 1000)
        self.ckpt_best_queue = queue.Queue(maxsize=train_config.get("keep_last_ckpts", 1))  #maintain a queue for best checkpoints
        self.ckpt_queue = queue.Queue(maxsize=1)

        #setup evaluation metrics and early stopping criteria
        self.eval_metric = train_config.get("eval_metric", "dtw").lower()
        if self.eval_metric not in ['bleu', 'chrf', "dtw"]:
            raise ConfigurationError("Invalid setting for 'eval_metric', "
                                     "valid options: 'bleu', 'chrf', 'DTW'")

        self.early_stopping_metric = train_config.get("early_stopping_metric", "eval_metric") #choose which metric to use for early stopping

        if self.early_stopping_metric in ["loss", "dtw"]: #if using loss or dtw, we want lower values, so minimize=true
            self.minimize_metric = True
        else:
            raise ConfigurationError("Invalid setting for 'early_stopping_metric', "
                                    "valid options: 'loss', 'dtw',.")

        #learning rate scheduling (plateau)
        self.scheduler, self.scheduler_step_at = build_scheduler(
            config=train_config,
            scheduler_mode="min" if self.minimize_metric else "max",
            optimizer=self.optimizer,
            hidden_size=config["model"]["encoder"]["hidden_size"])

        #data and batch handling
        self.level = "word"  #tokenization level
        self.shuffle = train_config.get("shuffle", True)  #shuffling enabled
        self.epochs = train_config["epochs"]
        self.batch_size = train_config["batch_size"]  #for both training and evaluation
        self.batch_type = "sentence"
        self.eval_batch_size = train_config.get("eval_batch_size",self.batch_size)
        self.eval_batch_type = train_config.get("eval_batch_type",self.batch_type)
        self.batch_multiplier = train_config.get("batch_multiplier", 1)  #accumulate gradient over 1 batch only (no gradient accumulation)

        #decoding setup
        self.max_output_length = train_config.get("max_output_length", None)  #max length for decoder output(used in sequence generation)

        #device setup
        self.device = torch.device("cuda" if train_config["use_cuda"] and torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.loss.to(self.device)

        # initialize training statistics
        self.steps = 0
        # stop training if this flag is True by reaching learning rate minimum
        self.stop = False
        self.total_tokens = 0
        self.best_ckpt_iteration = 0
        # initial values for best scores
        self.best_ckpt_score = np.inf if self.minimize_metric else -np.inf
        # comparison function for scores
        self.is_best = lambda score: score < self.best_ckpt_score \
            if self.minimize_metric else score > self.best_ckpt_score

        ## Checkpoint restart
        # If continuing
        #if model_continue:
            # Get the latest checkpoint
        #    ckpt = get_latest_checkpoint(model_dir)
        #    if ckpt is None:
        #        self.logger.info("Can't find checkpoint in directory %s", ckpt)
        #    else:
        #        self.logger.info("Continuing model from %s", ckpt)
        #        self.init_from_checkpoint(ckpt)

        # Skip frames
        self.skip_frames = config["data"].get("skip_frames", 1)




    #function to save a checkpoint
    def _save_checkpoint(self, type="every") -> None:
        # Define model path
        model_path = "{}/{}_{}.ckpt".format(self.model_dir, self.steps, type)
        # Define State
        state = {
            "steps": self.steps,
            "total_tokens": self.total_tokens,
            "best_ckpt_score": self.best_ckpt_score,
            "best_ckpt_iteration": self.best_ckpt_iteration,
            "model_state": self.model.state_dict(),
            "optimizer_state": self.optimizer.state_dict(),
            "scheduler_state": self.scheduler.state_dict() if \
            self.scheduler is not None else None,
        }
        #save
        torch.save(state, model_path)

        #if this is the best checkpoint, add it to the best checkpoint queue
        if type == "best":
            if self.ckpt_best_queue.full():
                to_delete = self.ckpt_best_queue.get()  # delete oldest ckpt
                try:
                    os.remove(to_delete)
                except FileNotFoundError:
                    self.logger.warning("Wanted to delete old checkpoint %s but "
                                        "file does not exist.", to_delete)

            self.ckpt_best_queue.put(model_path)

            best_path = "{}/best.ckpt".format(self.model_dir)
            try:
                # create/modify symbolic link for best checkpoint
                symlink_update("{}_best.ckpt".format(self.steps), best_path)
            except OSError:
                # overwrite best.ckpt
                torch.save(state, best_path)

        #if this is just the checkpoint at every validation
        elif type == "every":
            if self.ckpt_queue.full():
                to_delete = self.ckpt_queue.get()  # delete oldest ckpt
                try:
                    os.remove(to_delete)
                except FileNotFoundError:
                    self.logger.warning("Wanted to delete old checkpoint %s but "
                                        "file does not exist.", to_delete)

            self.ckpt_queue.put(model_path)

            every_path = "{}/every.ckpt".format(self.model_dir)
            try:
                # create/modify symbolic link for best checkpoint
                symlink_update("{}_best.ckpt".format(self.steps), every_path)
            except OSError:
                # overwrite every.ckpt
                torch.save(state, every_path)



    #function to initialise from a checkpoint
    def init_from_checkpoint(self, path: str) -> None:
        # Find last checkpoint
        model_checkpoint = load_checkpoint(path=path, use_cuda=self.use_cuda)

        # restore model and optimizer parameters
        self.model.load_state_dict(model_checkpoint["model_state"])
        self.optimizer.load_state_dict(model_checkpoint["optimizer_state"])

        if model_checkpoint["scheduler_state"] is not None and \
                self.scheduler is not None:
            # Load the scheduler state
            self.scheduler.load_state_dict(model_checkpoint["scheduler_state"])

        # restore counts
        self.steps = model_checkpoint["steps"]
        self.total_tokens = model_checkpoint["total_tokens"]
        self.best_ckpt_score = model_checkpoint["best_ckpt_score"]
        self.best_ckpt_iteration = model_checkpoint["best_ckpt_iteration"]

        self.model.to(self.device)



    #train and validate function
    def train_and_validate(self, train_data: Dataset, valid_data: Dataset) -> None:
        #make training iterator

        train_iter= torchtext_compat.make_data_iterator(train_data,      #used to iterate through batches of processed (gloss,skels,file) triplets
                                       train_data.fields,
                                       batch_size= self.batch_size,
                                       shuffle= self.shuffle,
                                       trg_pad_val=TARGET_PAD)
        #print(">>> Number of examples in train_data:", len(train_data))
        #print(">>> Fields keys:", train_data.fields.keys())
        #print(">>> Example 0 trg:", train_data[0].trg if len(train_data) > 0 else "Empty")

        #print("DEBUG vocab at iterator creation:", train_data.fields["src"].vocab)

        val_step=0

        #training loop
        for epoch_no in range(0,self.epochs):
            #self.logger.info("EPOCH %d", epoch_no + 1)

            if self.scheduler is not None and self.scheduler_step_at == "epoch":  #if scheduler is set to step on epoch, step the lr scheduler at the start of each epoch
                self.scheduler.step(epoch=epoch_no)

            self.model.train()

            # Reset statistics for each epoch.
            start = time.time()
            total_valid_duration = 0
            start_tokens = self.total_tokens
            count = self.batch_multiplier - 1
            epoch_loss = 0

            #loop through each batch
            for batch in iter(train_iter):
                #print(">>> First batch keys:", batch.keys())
                # reactivate training
                self.model.train()

                # create a Batch object from custom batch class
                batch = Batch(batch_dict=batch,
                              pad_index=self.pad_index,
                              target_pad=self.target_pad)

                update= count==0  #if update is true(count-tracks batch_multiplier value), gradients will be stepped this batch

                # Train the model on a batch
                batch_loss = self._train_batch(batch, update=update)
                self.tb_writer.add_scalar("train/train_batch_loss", batch_loss, self.steps)

                count = self.batch_multiplier if update else count
                count -= 1
                epoch_loss += batch_loss.detach().cpu().numpy()

                if self.scheduler is not None and self.scheduler_step_at == "step" and update:
                    self.scheduler.step()

                # log learning progress
                if self.steps % self.logging_freq == 0 and update:
                    elapsed = time.time() - start - total_valid_duration
                    elapsed_tokens = self.total_tokens - start_tokens
                    self.logger.info(
                        "Epoch %d | Step: %d | Batch Loss: %.6f | "
                        "Tokens per Sec: %.0f | Lr: %.6f",
                        epoch_no + 1, self.steps, batch_loss,
                        elapsed_tokens / elapsed,
                        self.optimizer.param_groups[0]["lr"])
                    start = time.time()
                    total_valid_duration = 0
                    start_tokens = self.total_tokens

                #validation block- runs after every validation_freq steps
                if self.steps % self.validation_freq == 0 and update:
                    valid_start_time = time.time()

                    valid_score, valid_loss, valid_references, valid_hypotheses, \
                        valid_inputs, all_dtw_scores, valid_file_paths, valid_bt_f1, valid_mpjpe,_,_,_,_,_,_= \
                        validate_on_data(
                            batch_size=self.eval_batch_size,
                            data=valid_data,
                            eval_metric=self.eval_metric,
                            model=self.model,
                            max_output_length=self.max_output_length,
                            loss_function=self.loss,
                            batch_type=self.eval_batch_type,
                            type="val",
                        )

                    val_step+=1

                     # Tensorboard writer
                    self.tb_writer.add_scalar("Valid/loss", valid_loss, self.steps)
                    self.tb_writer.add_scalar("Valid/dtw", valid_score, self.steps)
                    self.tb_writer.add_scalar("Valid/bt_f1", valid_bt_f1, self.steps)
                    self.tb_writer.add_scalar("Valid/mpjpe", valid_mpjpe, self.steps)


                    if self.early_stopping_metric == "loss":
                        ckpt_score = valid_loss
                    elif self.early_stopping_metric == "dtw":
                        ckpt_score = valid_score
                    else:
                        ckpt_score = valid_score

                    #if this validation is better than all previous ones, save as best checkpoint
                    new_best = False
                    self.best = False
                    if self.is_best(ckpt_score):
                        self.best = True
                        self.best_ckpt_score = ckpt_score
                        self.best_ckpt_iteration = self.steps
                        self.logger.info(
                            'Hooray! New best validation result [%s]!',
                            self.early_stopping_metric)
                        if self.ckpt_queue.maxsize > 0:
                            self.logger.info("Saving new checkpoint.")
                            new_best = True
                            self._save_checkpoint(type="best")

                        # generate video samples
                        display = list(range(0, len(valid_hypotheses), int(np.ceil(len(valid_hypotheses) / 13.15))))
                        self.produce_validation_video(
                            output_joints=valid_hypotheses,
                            inputs=valid_inputs,
                            references=valid_references,
                            model_dir=self.model_dir,
                            steps=self.steps,
                            display=display,
                            type="val_inf",
                            file_paths=valid_file_paths,
                        )


                    #save checkpoint after every validation step even if it is not the best
                    self._save_checkpoint(type="every")

                    if self.scheduler is not None and self.scheduler_step_at == "validation":
                        self.scheduler.step(ckpt_score)

                    # append to validation report
                    self._add_report(
                        valid_score=valid_score, valid_loss=valid_loss,
                        eval_metric=self.eval_metric,
                        new_best=new_best, report_type="val",)

                    valid_duration = time.time() - valid_start_time
                    total_valid_duration += valid_duration
                    self.logger.info('Validation result at epoch %d\n'
                                    '\t\t\tStep: %d | DTW: %.4f | BT-F1: %.4f | MPJPE: %.4f | Loss: %.4f | Duration: %.2fs',
                                    epoch_no + 1, self.steps, valid_score, valid_bt_f1, valid_mpjpe, valid_loss, valid_duration
                                    )


                if self.stop:
                    break

            if self.stop:
                self.logger.info(
                    'Training ended since minimum lr %f was reached.',
                     self.learning_rate_min)
                break

            if epoch_no % 20 == 0:
                self.logger.info('Epoch %d: Total training loss %.5f', epoch_no,
                                 epoch_loss)

        else:
            self.logger.info('Training ended after %d epochs.', epoch_no+1)


        self.logger.info('Best validation result at step %d: %.2f %s.',
                         self.best_ckpt_iteration, self.best_ckpt_score,
                         self.early_stopping_metric)

        # Save final model weights
        final_ckpt_path = os.path.join("./Checkpoints", "final_model.pth")
        torch.save(self.model.state_dict(), final_ckpt_path)
        self.logger.info("Final model saved to %s", final_ckpt_path)

        self.tb_writer.close()  # close Tensorboard writer



    #function to generate validation videos (side by side comparision of output and ground truth)
    def produce_validation_video(self, output_joints, inputs, references, display,
                                 model_dir, type, steps="", file_paths=None, dtw_file= None):

        # If not at test
        if type != "test":
            dir_name = model_dir + "/videos/Step_{}/".format(steps)
            if not os.path.exists(model_dir + "/videos/"):
                os.mkdir(model_dir + "/videos/")

        # If at test time
        elif type == "test":
            dir_name = model_dir + "/test_videos/"

        # Create model video folder if not exist
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)

        #generate one video per sample index in display
        for i in display:
            #pull the relevant data
            seq = output_joints[i]
            ref_seq = references[i]
            input = inputs[i]

            #create gloss label
            gloss_label = input[0]
            if input[1] != "</s>":  #skip eos tokens
                gloss_label += "_" + input[1]
            if input[2] != "</s>":
                gloss_label += "_" + input[2]

            # Alter the dtw timing of the produced sequence using the reference sequence, and collect the DTW score
            timing_hyp_seq, ref_seq_count, dtw_score = alter_DTW_timing(seq, ref_seq)

            #name the file using gloss and score
            video_ext = "{}_{}.mp4".format(gloss_label, "{0:.2f}".format(float(dtw_score)).replace(".", "_"))

            #get sequence id to track source files
            if file_paths is not None:
                sequence_ID = file_paths[i]
            else:
                sequence_ID = None

            #print and log dtw score
            print(sequence_ID + '    dtw: ' + '{0:.2f}'.format(float(dtw_score)))
            if dtw_file != None:
                dtw_file.writelines(sequence_ID + ' ' + '{0:.2f}'.format(float(dtw_score)) + '\n')

            #plot the video
            os.makedirs(dir_name, exist_ok=True)
            plot_video(joints=timing_hyp_seq,
                       file_path=dir_name,
                       video_name=None,
                       references=ref_seq_count,
                       skip_frames=self.skip_frames,
                       sequence_ID=sequence_ID)



    #function to save the skeletons
    def save_skels(self, output_joints, display, model_dir, type, file_paths=None):
        #create pickle output file
        picklefile = open(model_dir + "/phoenix14t.skels.%s" % type, "wb")

        #load metadata csv
        csvIn = pd.read_csv(model_dir + "/csv/%s_phoenix2014t.csv" % type, sep='|',encoding='utf-8')
        pickle_list = []

        #loop through selected samples
        for i in display:
            name = file_paths[i]
            #extract metadata
            video = name[len(os.path.dirname(name))+1:]
            signer = csvIn[csvIn['id']==video]['signer'].item()
            gloss = csvIn[csvIn['id']==video]['annotation'].item()
            text = csvIn[csvIn['id']==video]['translation'].item()
            #extract predicted joint sequences
            seq = output_joints[i].cpu()[:,:-1]
            sign = torch.tensor(seq, dtype = torch.float32)

            #build and store dictionary
            dict_num = {'name': name, 'signer': signer, 'gloss': gloss, 'text': text, 'sign': sign}

            pickle_list.append(dict_num)

        pickle.dump(pickle_list, picklefile)
        print("The skeletons of %s date have been save." % type)

    def save_predicted_skels_txt(self, output_joints, display, output_path):
        with open(output_path, "w", encoding="utf-8") as f:
           for i in display:
               seq = output_joints[i].cpu()
                # remove last dim if needed (e.g., 4D: x, y, z, conf → 3D)
               if seq.shape[-1] % 3 == 0 and seq.shape[-1] > 3:
                    seq = seq[:, :-1]
               flat = seq.contiguous().view(-1).tolist()
               f.write(" ".join(f"{x:.6f}" for x in flat) + "\n")
        print(f"[✔] Saved predicted skeletons to {output_path}")





    #function to train 1 batch
    def _train_batch(self, batch: Batch, update: bool = True) -> Tensor:
        #compute loss
        batch_loss = self.model.get_loss_for_batch(is_train=True,
                                                          batch=batch,
                                                          loss_function=self.loss)

        #determine what to divide the loss by (for normalisation)
        if self.normalization == "batch":
            normalizer = batch.nseqs
        elif self.normalization == "tokens":
            normalizer = batch.ntokens
        else:
            raise NotImplementedError("Only normalize by 'batch' or 'tokens'")


        norm_batch_loss= batch_loss/normalizer
        #divide further so that gradient is accumulated over batch_multiplier
        norm_batch_multiply= norm_batch_loss/self.batch_multiplier

        #backpropagate
        norm_batch_multiply.backward()


        #if gradient clipping function is defined, it is applied to prevent exploding gradients
        if self.clip_grad_fun is not None:
            self.clip_grad_fun(params=self.model.parameters())

        #update parameters
        if update:
            self.optimizer.step()
            self.optimizer.zero_grad()

            # increment step counter
            self.steps += 1

        #update total count token
        self.total_tokens += batch.ntokens

        return norm_batch_loss

    #function to log validation performance after an evaluation step
    def _add_report(self,valid_score: float, valid_loss: float, eval_metric: str, new_best: bool= False, report_type:str="val")->None:
        current_lr=-1;
        #extract the learning rate from the current parameter group
        for param_group in self.optimizer.param_groups:
            current_lr = param_group['lr']

        #check for early stopping
        if current_lr < self.learning_rate_min:
            self.stop = True

        #log to the validation report file
        if report_type == "val":
            with open(self.valid_report_file, 'a') as opened_file:
                opened_file.write(
                    "Steps: {} Loss: {:.5f}| DTW: {:.3f}|"
                    " LR: {:.6f} {}\n".format(
                        self.steps, valid_loss, valid_score,
                        current_lr, "*" if new_best else ""))


    #function to write all model parameters to the log
    def _log_parameters_list(self)->None:
        #filter only those parameters whose values are updated during training (trainable parameters)
        model_parameters = filter(lambda p: p.requires_grad,
                                  self.model.parameters())

        n_params = sum([np.prod(p.size()) for p in model_parameters])
        self.logger.info("Total params: %d", n_params)

        #collect the names of all the trainable parameters
        trainable_params = [n for (n, p) in self.model.named_parameters()
                            if p.requires_grad]

        self.logger.info("Trainable parameters: %s", sorted(trainable_params))
        assert trainable_params  #throws assertion error if no trainable parameters found


# Train function

In [9]:
def train (cfg_file:str, ckpt):
    cfg= load_config(cfg_file)  #cfg is a dictionary of all the configurations from the yaml file
    set_seed(seed=cfg["training"].get("random_seed", 42)) #for reproducibility

    #prepare the dataset and load it using the functions in data.py
    train_data, dev_data, test_data, src_vocab, trg_vocab = load_data(cfg=cfg)

    #print("train_data sample:")
    #print("src:", train_data[0].src)
    #print("trg:", train_data[0].trg)
    #print("file_paths:",train_data[0].file_paths)

    #instantiate the model
    model = build_model(cfg=cfg, src_vocab=src_vocab, trg_vocab=trg_vocab)

    #if there are checkpoints, load parameters from there instead of initialising
    if ckpt is not None and os.path.getsize(ckpt)>0:
        device= torch.device('cuda' if torch.cuda.is_available() and cfg["training"].get("use_cuda", True) else 'cpu')
        model_checkpoint= torch.load(ckpt, map_location= device)
        #model.load_state_dict(model_checkpoint["model_state"])
        model.load_state_dict(torch.load(ckpt, map_location=device))

    #initialise the train manager class
    trainer = TrainManager(config=cfg, model=model, test=False)

    #store a copy of the training config before training loop
    shutil.copy2(cfg_file, trainer.model_dir+"/Sign-IDD.yaml")
    log_cfg(cfg, trainer.logger)

    trainer.train_and_validate(train_data=train_data, valid_data=dev_data)

In [ ]:
train(cfg_file= config_path, ckpt=ckpt)

Output hidden; open in https://colab.research.google.com to view.

# Test function

In [ ]:
def test(cfg_file, ckpt: str):
    cfg = load_config(cfg_file)
    model_dir = cfg["training"]["model_dir"]

    batch_size = cfg["training"].get("eval_batch_size", cfg["training"]["batch_size"])
    batch_type = cfg["training"].get("eval_batch_type", cfg["training"].get("batch_type", "sentence"))
    use_cuda = cfg["training"].get("use_cuda", True)
    eval_metric = cfg["training"]["eval_metric"]
    max_output_length = cfg["training"].get("max_output_length", None)

    # create datasets for model input
    _, dev_data, test_data, src_vocab, trg_vocab = load_data(cfg=cfg)
    data_to_predict = {"dev": dev_data, "test": test_data}

    # load model architecture
    trained_model = build_model(cfg, src_vocab, trg_vocab)
    # load model weights
    trained_model.load_state_dict(
        torch.load("/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Checkpoints/final_model.pth", map_location=device)
    )

    trainer = TrainManager(model=trained_model, config=cfg, test=True)

    # choose the split (dev or test)
    for data_set_name, data_set in data_to_predict.items():
        print(f"\n--- Running inference on {data_set_name.upper()} set ---")

        # validate
        valid_score, valid_loss, valid_references, valid_hypotheses, \
        valid_inputs, all_dtw_scores, valid_file_paths, valid_bt_f1, valid_mpjpe, \
        valid_bleu1, valid_bleu4, valid_rouge_l, valid_wer, valid_fid, valid_mpjae = \
            validate_on_data(
                model=trained_model,
                data=data_set,
                batch_size=batch_size,
                max_output_length=max_output_length,
                eval_metric=eval_metric,
                loss_function=None,
                batch_type=batch_type,
                type="val" if data_set_name != "train" else "train_inf"
            )

        # save metrics to text file
        result_file = os.path.join(model_dir, f"{data_set_name}_metrics.txt")
        with open(result_file, 'w') as f:
            f.write(f"\n===== Evaluation on {data_set_name.upper()} set =====\n")
            f.write(f"DTW Score     : {np.mean(all_dtw_scores):.3f}\n")
            f.write(f"BT F1 Score   : {valid_bt_f1:.3f}\n")
            f.write(f"MPJPE         : {valid_mpjpe:.3f}\n")
            f.write(f"BLEU-1 Score  : {valid_bleu1:.3f}\n")
            f.write(f"BLEU-4 Score  : {valid_bleu4:.3f}\n")
            f.write(f"ROUGE-L Score : {valid_rouge_l:.3f}\n")
            f.write(f"WER Score     : {valid_wer:.3f}\n")
            f.write(f"FID Score     : {valid_fid:.3f}\n")
            f.write(f"MPJAE         : {valid_mpjae:.3f}\n")

        print(open(result_file).read())

        # save predicted skels in the same format as the input skels
        display = list(range(len(valid_hypotheses)))
        skels_output_path = os.path.join(model_dir, f"{data_set_name}.pred.skels")
        trainer.save_predicted_skels_txt(
            output_joints=valid_hypotheses,
            display=display,
            output_path=skels_output_path
        )

        # generate videos
        trainer.produce_validation_video(
            output_joints=valid_hypotheses,
            inputs=valid_inputs,
            references=valid_references,
            model_dir=model_dir,
            display=display,
            type=data_set_name,
            file_paths=valid_file_paths,
            dtw_file=open(result_file, 'a'),
        )


In [ ]:
test(cfg_file=config_path, ckpt=ckpt)

Output hidden; open in https://colab.research.google.com to view.

# Test with NLP eval metrics

In [10]:
from eval_helper import compute_bleu_scores, compute_rouge, compute_wer
import os

def evaluate_gloss_file(pred_gloss_path, reference_inputs, reference_file_paths):
    # Load predictions as {id: predicted gloss string}
    preds = {}
    with open(pred_gloss_path, 'r') as f:
        for line in f:
            sample_id, gloss = line.strip().split('|', 1)
            preds[sample_id] = gloss

    # Convert reference inputs to strings
    refs = [' '.join(tokens) for tokens in reference_inputs]

    # Map predictions to correct sample order using file_paths
    hyp_texts = []
    for path in reference_file_paths:
        sample_id = os.path.basename(path).replace('.pkl', '')
        hyp_texts.append(preds.get(sample_id, ''))

    # Compute metrics
    bleu1, bleu4 = compute_bleu_scores(refs, hyp_texts)
    rouge_l = compute_rouge(refs, hyp_texts)
    wer = compute_wer(refs, hyp_texts)

    return bleu1, bleu4, rouge_l, wer

### Modified validate_on_data

In [16]:
import torch
import numpy as np

import torchtext_compat
from data import Dataset, make_data_iterator
from torchtext_compat import Batch
from helpers import calculate_dtw
from eval_helper import calculate_bt_and_mpjpe, compute_rouge, compute_bleu_scores, compute_wer, compute_fid, mpjae
from model import Model
from constants import PAD_TOKEN, TARGET_PAD


# Validate epoch given a dataset
def validate_on_data2(model: Model,
                     data: Dataset,
                     batch_size: int,
                     max_output_length: int,
                     eval_metric: str,
                     loss_function: torch.nn.Module = None,
                     batch_type: str = "sentence",
                     type = "val",
                     BT_model = None
                     ):

    valid_iter = make_data_iterator(
        dataset=data, fields=data.fields, batch_size=batch_size,
        shuffle=True)

    pad_index = model.src_vocab.stoi[PAD_TOKEN]
    # disable dropout
    model.eval()
    # don't track gradients during validation
    with torch.no_grad():
        valid_hypotheses = []
        valid_references = []
        valid_inputs = []
        file_paths = []
        all_dtw_scores = []

        valid_loss = 0
        total_ntokens = 0
        total_nseqs = 0

        batches = 0
        for valid_batch in iter(valid_iter):
            # Extract batch
            batch = Batch(batch_dict=valid_batch, pad_index=pad_index, target_pad= TARGET_PAD)
            targets = batch.trg_input

            # run as during training with teacher forcing
            if loss_function is not None and batch.trg is not None:
                # Get the loss for this batch
                batch_loss = model.get_loss_for_batch(is_train=True,
                                                         batch=batch,
                                                         loss_function=loss_function)

                valid_loss += batch_loss
                total_ntokens += batch.ntokens
                total_nseqs += batch.nseqs

            output = model.forward(src=batch.src,
                                       trg_input=batch.trg_input[:, :, :150],
                                       src_mask=batch.src_mask,
                                       src_lengths=batch.src_lengths,
                                       trg_mask=batch.trg_mask,
                                       is_train=False)

            output = torch.cat((output, batch.trg_input[:, :, 150:]), dim=-1)

            # Add references, hypotheses and file paths to list
            valid_references.extend(targets)
            valid_hypotheses.extend(output)
            file_paths.extend(batch.file_paths)
            # Add the source sentences to list, by using the model source vocab and batch indices
            valid_inputs.extend([[model.src_vocab.itos[batch.src[i][j]] for j in range(len(batch.src[i]))] for i in
                                 range(len(batch.src))])

            # Calculate the full Dynamic Time Warping score - for evaluation
            dtw_score = calculate_dtw(targets, output)
            all_dtw_scores.extend(dtw_score)

            #Calculate bt and mpjpe
            bt_f1_scores, mpjpe_scores = calculate_bt_and_mpjpe(targets, output)

            # Can set to only run a few batches
            # if batches == math.ceil(20/batch_size):
            #     break
            batches += 1

        # Dynamic Time Warping scores
        current_valid_score = np.mean(all_dtw_scores)
        #Mean bt and mpjpe scores
        mean_bt_f1 = np.mean(bt_f1_scores)
        mean_mpjpe = np.mean(mpjpe_scores)

        #for split_name in ["dev", "test"]:
            #pred_file = f"/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Data/SLT_predicted.{split_name}.gloss"

        pred_file = f"/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Data/SLT_predicted.{type}.gloss"
        bleu1, bleu4, rouge_l, wer = evaluate_gloss_file(
                                            pred_file, valid_inputs, file_paths
                                           )

        # print(f"\n=== {type.upper()} Gloss Evaluation ===")
        # print(f"BLEU-1     : {bleu1:.3f}")
        # print(f"BLEU-4     : {bleu4:.3f}")
        # print(f"ROUGE-L    : {rouge_l:.3f}")
        # print(f"WER        : {wer:.3f}")


        # Convert token lists to text for NLP metrics
        #ref_texts = [' '.join(tokens) for tokens in valid_inputs]
        #hyp_texts = ref_texts  # Replace this with actual model outputs if you have textual predictions

        # NLP metrics
        #bleu1, bleu4 = compute_bleu_scores(ref_texts, hyp_texts)
        #rouge_l = compute_rouge(ref_texts, hyp_texts)
        #print("Rouge done")
        #rouge_l=0
        #wer_score = compute_wer(ref_texts, hyp_texts)
        #print("wer done")

        # FID and MPJAE using skeletal predictions
        #print("Hypotheses:", len(valid_hypotheses), valid_hypotheses[0].shape)
        #print("References:", len(valid_references), valid_references[0].shape)

        # Check memory size
        #import psutil
        #import os
        #print("RAM usage before FID:", psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2, "MB")


        fake_feats = torch.stack(valid_hypotheses).reshape(len(valid_hypotheses), -1).cpu().numpy()
        real_feats = torch.stack(valid_references).reshape(len(valid_references), -1).cpu().numpy()
        fid_score = compute_fid(fake_feats, real_feats)
        #print("fid done")
        mpjae_score = mpjae(torch.stack(valid_hypotheses), torch.stack(valid_references))
        #print("mpjae done")


    return current_valid_score, valid_loss, valid_references, valid_hypotheses, \
       valid_inputs, all_dtw_scores, file_paths, mean_bt_f1, mean_mpjpe, \
       bleu1, bleu4, rouge_l, wer, fid_score, mpjae_score

### Final test function

In [12]:
def test2(cfg_file, ckpt: str):
    cfg = load_config(cfg_file)
    model_dir = cfg["training"]["model_dir"]

    batch_size = cfg["training"].get("eval_batch_size", cfg["training"]["batch_size"])
    batch_type = cfg["training"].get("eval_batch_type", cfg["training"].get("batch_type", "sentence"))
    use_cuda = cfg["training"].get("use_cuda", True)
    eval_metric = cfg["training"]["eval_metric"]
    max_output_length = cfg["training"].get("max_output_length", None)

    _, dev_data, test_data, src_vocab, trg_vocab = load_data(cfg=cfg)
    data_to_predict = {"dev": dev_data, "test": test_data}

    trained_model = build_model(cfg, src_vocab, trg_vocab)
    trained_model.load_state_dict(
        torch.load("/content/drive/MyDrive/Sign IDD Openpose/Sign-IDD/Checkpoints/final_model.pth", map_location=device)
    )

    trainer = TrainManager(model=trained_model, config=cfg, test=True)

    for data_set_name, data_set in data_to_predict.items():
        print(f"\n--- Running inference on {data_set_name.upper()} set ---")

        valid_score, valid_loss, valid_references, valid_hypotheses, \
        valid_inputs, all_dtw_scores, valid_file_paths, valid_bt_f1, valid_mpjpe, \
        valid_bleu1, valid_bleu4, valid_rouge_l, valid_wer, valid_fid, valid_mpjae = \
            validate_on_data2(
                model=trained_model,
                data=data_set,
                batch_size=batch_size,
                max_output_length=max_output_length,
                eval_metric=eval_metric,
                loss_function=None,
                batch_type=batch_type,
                type=data_set_name,
            )

        # Save metrics to file
        result_file = os.path.join(model_dir, f"{data_set_name}_metrics.txt")
        with open(result_file, 'w') as f:
            f.write(f"\n===== Evaluation on {data_set_name.upper()} set =====\n")
            f.write(f"DTW Score     : {np.mean(all_dtw_scores):.3f}\n")
            f.write(f"BT F1 Score   : {valid_bt_f1:.3f}\n")
            f.write(f"MPJPE         : {valid_mpjpe:.3f}\n")
            f.write(f"BLEU-1 Score  : {valid_bleu1:.3f}\n")
            f.write(f"BLEU-4 Score  : {valid_bleu4:.3f}\n")
            f.write(f"ROUGE-L Score : {valid_rouge_l:.3f}\n")
            f.write(f"WER Score     : {valid_wer:.3f}\n")
            f.write(f"FID Score     : {valid_fid:.3f}\n")
            f.write(f"MPJAE         : {valid_mpjae:.3f}\n")

        print(open(result_file).read())

        # Save predicted skels in correct format
        display = list(range(len(valid_hypotheses)))
        skels_output_path = os.path.join(model_dir, f"{data_set_name}.pred.skels")
        trainer.save_predicted_skels_txt(
            output_joints=valid_hypotheses,
            display=display,
            output_path=skels_output_path
        )

        # Optional: Generate videos
        trainer.produce_validation_video(
            output_joints=valid_hypotheses,
            inputs=valid_inputs,
            references=valid_references,
            model_dir=model_dir,
            display=display,
            type=data_set_name,
            file_paths=valid_file_paths,
            dtw_file=open(result_file, 'a'),
        )


In [17]:
test2(cfg_file=config_path,ckpt=ckpt)

Output hidden; open in https://colab.research.google.com to view.

# **DATA FLOW**

## Dataset preparation (load_data)
- glosses are tokenized, src_vocab is created (.src)
- skels are converted into a list of N(number of frames) 1d tensors of size 151 (150 keypoints + 1 frame counter)-> N frames with 150 keypoints each (.trg)
- files (mp4 paths)- no processing (.file_paths)
- each element in train_data is a triplet (.src, .trg, .file_path)

## Preprocessing and batching (make_data_iter)
- src - convert gloss tokens into indices (tensors) using vocab, group tensors based on batch size
- trg - each element has a list of 2d tensors (frames, [151]), pads all sequences in the batch acc to the sequence with max no of frames and returns a 3d tensor (batch size, frames, [151])
- file_paths - no changes
- each batch object is a dictionary of the above 3 preprocessed fields

## Training
- glosses - embedded and passed through transformer encoder layer
- target poses(skels)
      - passed through acd
      - id is applied for better structural representation of the poses  (3d joints are converted into features- 7d vector per bone (3d coordinates+bonelength+3d unit direction)
      - diffusion (noise is added to ground truth poses)
- acd denoiser uses encoded glosses to reverse noise in the ground truth poses and produces pred_poses
- loss is calculated by comparing the original ground truth poses (without noise) and pred_poses

## Validation
- dtw scores are calculated using ground truth and predicted poses (unseen data)
- videos are rendered for best predictions (lower dtw)
- b1, b4, rouge and wer metrics are calculated using the predicted glosses produced by the backtranslator Sign IDD SLT (converts the predicted poses of Sign IDD back to gloss sequences)
- additional metrics include fid, mpjpe, mpjae and bt f1


# Model architecture visualisation

In [ ]:
!{sys.executable} -m pip install torchview


In [ ]:
import torch.nn as nn
class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, inputs):
        return self.model(
            is_train=False,
            src=inputs[0],
            trg_input=inputs[1],
            src_mask=inputs[2],
            src_lengths=inputs[3],
            trg_mask=inputs[4]
        )


In [ ]:
from torchview import draw_graph
#from wrapped_model import WrappedModel  # make sure this class is defined

cfg= load_config(config_path)
_, _, _, src_vocab, trg_vocab = load_data(cfg=cfg)
model = build_model(cfg, src_vocab, trg_vocab).to(device)
wrapped_model = WrappedModel(model)

# Extract sizes from config or model
vocab_size = len(src_vocab)
src_len = 20               # example input sentence length (you can choose 20–40)
pose_len = 30              # number of frames (temporal dimension)
pose_dim = cfg["model"]["trg_size"]  # e.g., 151

# Create dummy inputs
dummy_src = torch.randint(0, vocab_size, (1, src_len)).to(device)
dummy_trg = torch.randn(1, pose_len, pose_dim).to(device)
dummy_src_mask = torch.ones(1, 1, src_len).bool().to(device)
dummy_src_lengths = torch.tensor([src_len]).to(device)
dummy_trg_mask = torch.ones(1, 1, pose_len).bool().to(device)

input_tuple = (dummy_src, dummy_trg, dummy_src_mask, dummy_src_lengths, dummy_trg_mask)

# Visualize the model
graph = draw_graph(
    wrapped_model,
    input_data=(input_tuple,),
    input_size=None,
    expand_nested=True,
    save_graph=True,
    directory=".",
    filename="model_architecture",
    roll=True
)

graph.visual_graph  # displays inside Jupyter


### 151st value??? counter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Path to your file
file_path = "./Data/train.skels"

# Read and parse the file
with open(file_path, 'r') as f:
    lines = f.readlines()

# Assume 151 values per frame
frame_dim = 151
feature_dim = 150

# Parse and visualize each line
for i, line in enumerate(lines):
    values = np.fromstring(line.strip(), sep=' ')
    num_frames = len(values) // frame_dim
    data = values.reshape(num_frames, frame_dim)

    features = data[:, :feature_dim]
    counters = data[:, -1]


    print(f"\nLine {i+1} - {num_frames} frames")
    print("Frame\tCounter\tSample Features")
    j=0;
    for f in range(num_frames):
        print(f"{f+1}\t{counters[f]}\t{features[f, :5]} ...")

    # Optional: Plot counter values across frames
    plt.figure(figsize=(8, 2))
    plt.plot(counters, marker='o')
    plt.title(f"Line {i+1} - Frame Counters")
    plt.xlabel("Frame")
    plt.ylabel("Counter")
    plt.grid(True)
    plt.show()


In [ ]:
# Load the file and read all numerical values into a list
file_path = "./Data/train.skels"

# Read the values from the file
with open(file_path, "r") as f:
    data = f.read()

# Convert the data into a list of floats
values = [float(x) for x in data.strip().split()]

# Extract every 151st value (multiples of 151, 1-based index)
multiples_of_151 = [values[i - 1] for i in range(151, len(values) + 1, 151)]

multiples_of_151[:86]  # Show first 86 such values as a sample
